In [14]:
from textblob import TextBlob
import tweepy
import sys
import numpy as np


In [15]:
#Verstappen 2015-2016,2016-2018,2019-2020, 2021
#Seb 2008-09, 2010-2013,2014-2018,2019-2020,2021
#Ham 2007-09, 2010-2013, 2014-2016,2017-2020, 2021

In [16]:
twkeys = open(r"twitterkeys.txt").read().splitlines() #waiting till twitter apikey is approved
tkey = twkeys[0]
tkeysecret = twkeys[1]
taccesstoken = twkeys[2]
taccesstokensecret = twkeys[3]


In [17]:
authhandler = tweepy.OAuthHandler(consumer_key=tkey,consumer_secret=tkeysecret)
authhandler.set_access_token(taccesstoken,taccesstokensecret)
api = tweepy.API(authhandler)

In [25]:
topic = np.array(['Lewis Hamilton','Sebastian Vettel','Max Verstappen'])
numtweets = 100000
tweets = []
for i in range(len(topic)):
    temp = tweepy.Cursor(api.search, q = topic[i], lang = 'en').items(numtweets) #textblob works best on english
    tweets.append(temp)